In [138]:
#setup import libs
import numpy as np
import pandas as pd

# various options in pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 3)

In [16]:
def import_file(File_Path, Local=False):
    """
    import file into DataFrame and strip leading and tailing 
    whitespaces if any.
    File_path: internet link(URL) or where file is stored
    Local: is it URL or File stored in local?, Defualt True(store in local)
    """
    import pandas as pd
    
    if Local:
        fh = open(File_Path)
    else:
        
        from urllib.request import urlopen  # the lib that handles the url stuff
        fh = urlopen(File_Path) # it's a file like object and works just like a file
        
    data = pd.read_csv(fh, header = 0) #import file as DataFrame
    
    fh.close()
        
    return data

In [72]:
#import_file('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190921.txt', False)
DF_19021 = import_file('http://web.mta.info/developers/data/nyct/turnstile/turnstile_190921.txt')
DF_19021.head(5)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,00:00:00,REGULAR,7198818,2438323
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,04:00:00,REGULAR,7198834,2438325
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,08:00:00,REGULAR,7198847,2438354
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,12:00:00,REGULAR,7198929,2438428
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/14/2019,16:00:00,REGULAR,7199125,2438483


In [73]:
DF_19021.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

In [109]:

DF_19021.rename(columns=lambda x: x.strip(), inplace=True)
DF_19021.sort_values(["STATION","SCP","DATE","TIME"], inplace=True)
df = DF_19021.groupby(["STATION", "SCP"])
DF_19021['ENTRIES_DIFF'] = DF_19021.groupby(["STATION", "SCP"])['ENTRIES'].diff()
DF_19021['EXITS_DIFF'] = DF_19021.groupby(["STATION", "SCP"])['EXITS'].diff()
# DF_19021['ENTRIES_DIFF'] = df['ENTRIES'].diff()
# DF_19021['EXITS_DIFF'] = df['EXITS'].diff()
# df['EXITS_DIFF'] = df['EXITS'].diff()

In [ ]:
(DF_19021.groupby()
         .diff()
        .sort_values()
     .rest_index())

In [110]:
DF_19021.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF
31610,H007,R248,00-00-00,1 AV,L,BMT,09/14/2019,00:00:00,REGULAR,14976068,16737583,NaN,NaN
31611,H007,R248,00-00-00,1 AV,L,BMT,09/14/2019,04:00:00,REGULAR,14976068,16737595,0.0,12.0
31612,H007,R248,00-00-00,1 AV,L,BMT,09/14/2019,08:00:00,REGULAR,14976068,16737610,0.0,15.0
31613,H007,R248,00-00-00,1 AV,L,BMT,09/14/2019,12:00:00,REGULAR,14976068,16737636,0.0,26.0
31614,H007,R248,00-00-00,1 AV,L,BMT,09/14/2019,16:00:00,REGULAR,14976068,16737653,0.0,17.0
31615,H007,R248,00-00-00,1 AV,L,BMT,09/14/2019,20:00:00,REGULAR,14976068,16737665,0.0,12.0
31616,H007,R248,00-00-00,1 AV,L,BMT,09/15/2019,00:00:00,REGULAR,14976068,16737677,0.0,12.0
31617,H007,R248,00-00-00,1 AV,L,BMT,09/15/2019,04:00:00,REGULAR,14976068,16737682,0.0,5.0
31618,H007,R248,00-00-00,1 AV,L,BMT,09/15/2019,08:00:00,REGULAR,14976068,16737703,0.0,21.0
31619,H007,R248,00-00-00,1 AV,L,BMT,09/15/2019,12:00:00,REGULAR,14976068,16737713,0.0,10.0


In [111]:
DF_19021.describe()

,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF
count,2.049e+05,2.049e+05,2.006e+05,2.006e+05
mean,4.211e+07,3.487e+07,2.138e+03,-2.000e+04
std,2.156e+08,1.995e+08,1.668e+08,1.620e+08
min,0.000e+00,0.000e+00,-2.023e+09,-2.062e+09
25%,2.898e+05,1.294e+05,1.000e+00,1.000e+00
50%,2.041e+06,1.179e+06,6.600e+01,4.800e+01
75%,6.619e+06,4.536e+06,3.170e+02,2.280e+02
max,2.129e+09,2.124e+09,2.023e+09,2.062e+09


In [112]:
x =DF_19021.count()
y = DF_19021[(DF_19021.EXITS_DIFF < 0) | (DF_19021.ENTRIES_DIFF < 0)].count()
x / y

C/A             7.724
UNIT            7.724
SCP             7.724
STATION         7.724
LINENAME        7.724
DIVISION        7.724
DATE            7.724
TIME            7.724
DESC            7.724
ENTRIES         7.724
EXITS           7.724
ENTRIES_DIFF    7.561
EXITS_DIFF      7.561
dtype: float64

In [127]:
DF_cleaned = DF_19021[(DF_19021.EXITS_DIFF >= 0) & (DF_19021.ENTRIES_DIFF >= 0)]



DF_cleaned = DF_cleaned[(DF_cleaned.EXITS_DIFF <= 4e4) & (DF_cleaned.ENTRIES_DIFF <= 4e4)]


In [132]:
DF_cleaned.groupby('STATION').max()

,C/A,UNIT,SCP,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF
STATION,,,,,,,,,,,,
1 AV,H008,R248,01-00-04,L,BMT,09/20/2019,20:00:00,REGULAR,955648374,998898952,1415.0,1659.0
103 ST-CORONA,R529,R208,00-06-01,7,IRT,09/20/2019,20:00:00,REGULAR,15796577,24865935,1426.0,1412.0
104 ST,N137,R354,00-06-01,JZ,IND,09/20/2019,20:00:00,REGULAR,1681122840,978344941,355.0,162.0
110 ST,R254,R181,01-00-02,6,IRT,09/20/2019,20:00:00,REGULAR,14169322,12812189,1004.0,948.0
111 ST,R530,R355,01-06-01,A,IRT,09/20/2019,21:00:00,REGULAR,50331822,16777216,939.0,803.0
...,...,...,...,...,...,...,...,...,...,...,...,...
WOODLAWN,R294,R052,00-00-04,4,IRT,09/20/2019,20:00:00,REGULAR,8428033,6877294,740.0,316.0
WORLD TRADE CTR,N094,R029,01-06-03,ACE23,IND,09/20/2019,20:00:00,REGULAR,1580049935,1829105876,1226.0,1402.0
WTC-CORTLANDT,R109,R305,05-03-00,1,IRT,09/20/2019,21:00:00,REGULAR,730833374,583315,877.0,653.0


In [130]:
DF_19021.count()

C/A             204928
UNIT            204928
SCP             204928
STATION         204928
LINENAME        204928
DIVISION        204928
DATE            204928
TIME            204928
DESC            204928
ENTRIES         204928
EXITS           204928
ENTRIES_DIFF    200608
EXITS_DIFF      200608
dtype: int64

In [131]:
DF_cleaned.count()

C/A             157532
UNIT            157532
SCP             157532
STATION         157532
LINENAME        157532
DIVISION        157532
DATE            157532
TIME            157532
DESC            157532
ENTRIES         157532
EXITS           157532
ENTRIES_DIFF    157532
EXITS_DIFF      157532
dtype: int64

In [143]:
DF_cleaned[DF_cleaned.STATION == '1 AV'].groupby(['STATION','DATE','TIME']).sum()


ENTRIES       EXITS  ENTRIES_DIFF  EXITS_DIFF
STATION DATE       TIME                                                      
1 AV    09/14/2019 04:00:00  1415291746  1465723388           3.0        46.0
                   08:00:00  1415291752  1465723464           6.0        76.0
                   12:00:00  1415291754  1465723546           2.0        82.0
                   16:00:00  1415291762  1465723626           8.0        80.0
                   20:00:00  1415291762  1465723696           0.0        70.0
        09/15/2019 00:00:00  1415291763  1465723730           1.0        34.0
                   04:00:00  1415291766  1465723757           3.0        27.0
                   08:00:00  1415291770  1465723814           4.0        57.0
                   12:00:00  1415291774  1465723865           4.0        51.0
                   16:00:00  1415291778  1465723936           4.0        71.0
                   20:00:00  1415291788  1465724004          10.0        68.0
        09/16/2019 00:00:00  1415291794  1465724050           6.0        46.0
                   04:00:00  1415291796  1465724081           2.0        31.0
                   08:00:00  1415293245  1465727134        1449.0      3053.0
                   12:00:00  1415297532  1465732263        4287.0      5129.0
                   16:00:00  1415301738  1465735682        4206.0      3419.0
                   20:00:00  1415307432  1465741381        5694.0      5699.0
        09/17/2019 00:00:00  1415309240  1465743000        1808.0      1619.0
                   04:00:00  1415309470  1465743182         230.0       182.0
                   08:00:00  1415310997  1465746427        1527.0      3245.0
                   12:00:00  1415315482  1465751604        4485.0      5177.0
                   16:00:00  1415319762  1465755177        4280.0      3573.0
                   20:00:00  1415325774  1465761141        6012.0      5964.0
        09/18/2019 00:00:00  1415328070  1465763244        2296.0      2103.0
                   04:00:00  1415328318  1465763445         248.0       201.0
                   08:00:00  1415329807  1465766601        1489.0      3156.0
                   12:00:00  1415334239  1465771932        4432.0      5331.0
                   16:00:00  1415338514  1465775528        4275.0      3596.0
                   20:00:00  1415344535  1465781514        6021.0      5986.0
        09/19/2019 00:00:00  1415346785  1465783637        2250.0      2123.0
                   04:00:00  1415347052  1465783826         267.0       189.0
                   08:00:00  1415348520  1465787033        1468.0      3207.0
                   12:00:00  1415353025  1465792260        4505.0      5227.0
                   16:00:00  1415357412  1465795783        4387.0      3523.0
                   20:00:00  1415363474  1465801962        6062.0      6179.0
        09/20/2019 00:00:00  1415365952  1465803982        2478.0      2020.0
                   04:00:00  1415366228  1465804243         276.0       261.0
                   08:00:00  1415367576  1465807220        1348.0      2977.0
                   12:00:00  1415372095  1465812485        4519.0      5265.0
                   16:00:00  1415376350  1465816444        4255.0      3959.0
                   20:00:00  1415382692  1465822806        6342.0      6362.0

In [147]:
DF_cleaned['DATE'] = pd.to_datetime(DF_cleaned['DATE'])

DF_cleaned['day_of_week'] = DF_cleaned['DATE'].dt.day_name()
DF_cleaned.head(50)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,ENTRIES_DIFF,EXITS_DIFF,day_of_week
31611,H007,R248,00-00-00,1 AV,L,BMT,2019-09-14,04:00:00,REGULAR,14976068,16737595,0.0,12.0,Saturday
31612,H007,R248,00-00-00,1 AV,L,BMT,2019-09-14,08:00:00,REGULAR,14976068,16737610,0.0,15.0,Saturday
31613,H007,R248,00-00-00,1 AV,L,BMT,2019-09-14,12:00:00,REGULAR,14976068,16737636,0.0,26.0,Saturday
31614,H007,R248,00-00-00,1 AV,L,BMT,2019-09-14,16:00:00,REGULAR,14976068,16737653,0.0,17.0,Saturday
31615,H007,R248,00-00-00,1 AV,L,BMT,2019-09-14,20:00:00,REGULAR,14976068,16737665,0.0,12.0,Saturday
31616,H007,R248,00-00-00,1 AV,L,BMT,2019-09-15,00:00:00,REGULAR,14976068,16737677,0.0,12.0,Sunday
31617,H007,R248,00-00-00,1 AV,L,BMT,2019-09-15,04:00:00,REGULAR,14976068,16737682,0.0,5.0,Sunday
31618,H007,R248,00-00-00,1 AV,L,BMT,2019-09-15,08:00:00,REGULAR,14976068,16737703,0.0,21.0,Sunday
31619,H007,R248,00-00-00,1 AV,L,BMT,2019-09-15,12:00:00,REGULAR,14976068,16737713,0.0,10.0,Sunday
31620,H007,R248,00-00-00,1 AV,L,BMT,2019-09-15,16:00:00,REGULAR,14976068,16737732,0.0,19.0,Sunday


In [156]:
list(DF_cleaned.groupby(['STATION']).sum().sort_values('ENTRIES_DIFF', ascending=False).index)



['14 ST-UNION SQ',
 'FULTON ST',
 '59 ST COLUMBUS',
 'TIMES SQ-42 ST',
 '34 ST-PENN STA',
 'FLUSHING-MAIN',
 '47-50 STS ROCK',
 'CHAMBERS ST',
 'GRD CNTRL-42 ST',
 'PATH NEW WTC',
 'JKSN HT-ROOSVLT',
 '34 ST-HERALD SQ',
 'W 4 ST-WASH SQ',
 'ATL AV-BARCLAY',
 '42 ST-PORT AUTH',
 '42 ST-BRYANT PK',
 'FT HAMILTON PKY',
 '14 ST',
 'JAY ST-METROTEC',
 '72 ST',
 'BOROUGH HALL',
 '86 ST',
 'JAMAICA CENTER',
 '57 ST-7 AV',
 '72 ST-2 AVE',
 'DELANCEY/ESSEX',
 'JOURNAL SQUARE',
 '161/YANKEE STAD',
 '23 ST',
 '59 ST',
 '86 ST-2 AVE',
 'FOREST HILLS 71',
 'CROWN HTS-UTICA',
 '125 ST',
 'SUTPHIN-ARCHER',
 'KEW GARDENS',
 'WALL ST',
 "B'WAY-LAFAYETTE",
 '68ST-HUNTER CO',
 '33 ST',
 'THIRTY THIRD ST',
 '5 AV/53 ST',
 '66 ST-LINCOLN',
 'BOWLING GREEN',
 'JUNCTION BLVD',
 'GROVE STREET',
 '3 AV-149 ST',
 '103 ST-CORONA',
 '96 ST',
 'BEDFORD AV',
 'JAMAICA 179 ST',
 '168 ST',
 '34 ST-HUDSON YD',
 'CORTLANDT ST',
 '96 ST-2 AVE',
 'BROOKLYN BRIDGE',
 'GRAND ST',
 'LEXINGTON AV/63',
 '51 ST',
 '49 ST',
 'C

In [21]:
# total number of missing values 
np.count_nonzero(DF_19021.isnull())

0

In [96]:
DF_cleaned

C/A             26531
UNIT            26531
SCP             26531
STATION         26531
LINENAME        26531
DIVISION        26531
DATE            26531
TIME            26531
DESC            26531
ENTRIES         26531
EXITS           26531
ENTRIES_DIFF    26531
EXITS_DIFF      26531
dtype: int64

## Info need to extract
__Find Stations w/ highest Entries and Exits__: But have to exclude locations w/ high tourists (compare between weekend and weekdays)  
__Narrow down which time in day has higher rate__  
__Find demographic info with Gender and industries base on zipcode?__


In [22]:
# Find Top 5 Max Exits and Entries
DF_19021.ENTRIES.max()

2129093303

In [15]:
#groupby station, find the mean for Entries and Exits
DF_19021_Station = DF_19021.groupby('STATION').agg(['mean','count'])
DF_19021_Station

ENTRIES        \
                      mean count   
STATION                            
1 AV             1.415e+08   420   
103 ST           9.585e+06   630   
103 ST-CORONA    8.163e+06   378   
104 ST           5.388e+08   378   
110 ST           6.482e+06   252   
...                    ...   ...   
WOODLAWN         6.258e+06   207   
WORLD TRADE CTR  8.880e+07   756   
WTC-CORTLANDT    2.101e+07  1470   
YORK ST          1.459e+07   126   
ZEREGA AV        3.520e+05   246   

                EXITS                                                                 \
                                                                                mean   
STATION                                                                                
1 AV                                                     1.466e+08                     
103 ST                                                   7.863e+06                     
103 ST-CORONA                                            8.149e+06                     
104 ST                                                   3.921e+08                     
110 ST                                                   6.195e+06                     
...                                                            ...                     
WOODLAWN                                                 2.427e+06                     
WORLD TRADE CTR                                          1.023e+08                     
WTC-CORTLANDT                                            9.607e+04                     
YORK ST                                                  1.168e+07                     
ZEREGA AV                                                1.903e+05                     

                       
                count  
STATION                
1 AV              420  
103 ST            630  
103 ST-CORONA     378  
104 ST            378  
110 ST            252  
...               ...  
WOODLAWN          207  
WORLD TRADE CTR   756  
WTC-CORTLANDT    1470  
YORK ST           126  
ZEREGA AV         246  

[378 rows x 4 columns]